In [105]:
import numpy as np, math
import argparse
import pickle
import matplotlib
matplotlib.use('agg')
from matplotlib import pyplot as plt

import os, sys
from tensorflow.examples.tutorials.mnist import input_data


alpha = 0.1
class NeuralNetwork:
    def __init__(self, *args, dropout=False):
        # Layers of the neural network (doesn't include an input layer)
        self.loss = []
        self.layers = []

        self.image_count = 0

        current_wire = None
        for i, (layerType, units) in enumerate(*args):
            # For each layer defined, create the layer and attach it to the output wires
            # of the preceding layer

            if i == 0:
                # Create wires going from inputs to first hidden layer
                current_wire = Wire(units)
            else:
                new_layer = {
                    'sigmoid': sigmoidLayer(current_wire, units, dropout=dropout),
                    'softmax': softmaxLayer(current_wire, units, dropout=dropout),
                    'relu': reluLayer(current_wire, units, dropout=dropout)
                }.get(layerType, None)
                current_wire = new_layer.outputWire
                self.layers.append(new_layer)

    def train_and_validate(self, train_data, validation_data, epochs):
        global alpha
        alpha = 0.01
        for epoch in range(epochs):
            print("Epoch:", epoch)
            self.train(train_data)
            self.test(validation_data)
            alpha *= 1

    def train(self, data):
        train_loss = []
        for i, (input, label) in enumerate(zip(data[0], data[1])):
            output = self.forward_pass(input, training=True)
            errorGradient = label-np.round(output)  # TODO: Modify this when gradient calculation is resolved
            print("training", "output:", output, "label:", label, "errorGradient:", errorGradient)
            train_loss.append(-label.T.dot(output))
            self.backward_pass(errorGradient)
        self.loss.append(sum(train_loss)/float(len(train_loss)))
        plt.clf()
        plt.plot(self.loss)
#         plt.savefig('images/training_' + str(self.image_count) + '.png')
        self.image_count += 1

    def test(self, data):
            correct_count = 0
            for i, (input, label) in enumerate(zip(data[0], data[1])):
                
                output = self.forward_pass(input)
                print (input, output, label)
                if np.round(output) == label:
                    correct_count += 1

            print(correct_count / data[0].shape[0])
            print()

    def forward_pass(self, input, training=False):
        current_output = input
        for i, layer in enumerate( self.layers ):
            current_output = layer.forward(current_output, training=training)
        return current_output

    def calculate_error(self, prediction, label):
        return  0.5*(prediction-label)**2 #TODO: Modify when gradient issue resolved

    def backward_pass(self, errorGradient):
        for i, layer in enumerate(self.layers[::-1]):
            if i == 0:
                layer.outputWire.gradients = errorGradient
                layer.backward() #TODO: Modify when gradient issue resolved
            else:
                layer.backward()

    def saveModel(self):
        pickle.dump( self, open ("model", "wb"))

    @staticmethod
    def load_model():
        return pickle.load(open("model", "rb"))


class Wire:
    def __init__(self, input_dimensions ):
        self.input_dimensions = input_dimensions

    def initialize( self, units, fn ):
        self.dropout_proba = 0.5
        self.units = units
        self.weights = fn(self.input_dimensions + 1, units)
        self.gradients = None
        self.velocity = np.zeros(self.weights.shape)

    def generate_dropout_variables(self):
        self.dropout_vector = np.random.choice([0, 1], size=(self.units, ), p=[self.dropout_proba, 1-self.dropout_proba])
        self.dropout_matrix = np.diag(self.dropout_vector)


class Layer:
    def __init__( self, inputWire, units, dropout=False, weight_fn = lambda x, y: np.random.randn(x, y) / np.sqrt(x)):
        if not inputWire:
            raise TypeError("Must initialize layer with Wire")
        self.dropout = dropout
        self.inputWire = inputWire
        self.inputWire.initialize(units, fn = weight_fn)

        self.outputWire = Wire(units)

    def forward( self, x, training=False):
        raise NotImplementedError("Must implement forward-pass function -- forward( self,  )")
    def backward( self ):
        raise NotImplementedError("Must implement backward-pass function -- backward( self,  )" )


class sigmoidLayer( Layer ):
    def __init__(self, inputWire, units, dropout=False):
        super().__init__(inputWire, units, dropout)

    def forward(self, x, training=False):
        self.inputs = x
        # Add input value of "1" to the input array for the bias weight (w_0*x_0 + ...)
        self.inputs = np.insert(x, 0, 1.0, 0)
        self.linear_outputs = self.inputWire.weights.T.dot(self.inputs)


        self.outputs = self.sigmoid(self.linear_outputs)
        if training and self.dropout:
            self.inputWire.generate_dropout_variables()
            # self.outputs = self.outputs.dot(self.inputWire.dropout_matrix)
            self.outputs = self.outputs*self.inputWire.dropout_vector / self.inputWire.dropout_proba
        return self.outputs

    def sigmoid( self, x ):
        # Calculate sigmoid on vector cells
        self.sig_values = 1 / (1 + np.exp(-x))
        return self.sig_values
        
    def backward( self ):
        d_p_wrt_f = self.sig_values * (1 - self.sig_values)
        if self.dropout:
            d_p_wrt_f = d_p_wrt_f*self.inputWire.dropout_vector
        self.d_E_wrt_f = np.sum(self.outputWire.gradients[1:], 1) * d_p_wrt_f if self.outputWire.gradients.shape[0] > 1 else self.outputWire.gradients * d_p_wrt_f
        d_E_wrt_x = self.inputWire.weights.dot(np.diag(self.d_E_wrt_f)) # Multiply outgoing gradients by the dE/df of their respective node in this layer
        self.inputWire.gradients = d_E_wrt_x
        # print("backward gradient:", d_E_wrt_x)

        self.update()

    def update( self ):
        global alpha
        # if self.dropout:
        #     self.inputWire.weights -= alpha*(np.outer(self.inputs, self.inputWire.dropout_matrix.dot(self.d_E_wrt_f)))
        # self.inputWire.velocity = self.inputWire.momentum*self.inputWire.velocity - alpha * (np.outer(self.inputs, self.d_E_wrt_f))
        # self.inputWire.weights += self.inputWire.velocity
        self.inputWire.weights -= alpha * (np.outer(self.inputs, self.d_E_wrt_f))
        
class reluLayer(Layer):
    def __init__(self, inputWire, units, dropout=False):
        super().__init__(inputWire, units, dropout, weight_fn=lambda x, y: np.random.randn(x, y) / np.sqrt(x/2))

    def forward(self, x, training=False):
        self.inputs = x
        # Add input value of "1" to the input array for the bias weight (w_0*x_0 + ...)
        self.inputs = np.insert(x, 0, 1.0, 0)
        self.linear_outputs = self.inputWire.weights.T.dot(self.inputs)

        self.outputs = self.relu(self.linear_outputs)
        if training and self.dropout:
            self.inputWire.generate_dropout_variables()
            # self.outputs = self.outputs.dot(self.inputWire.dropout_matrix)
            self.outputs = self.outputs * self.inputWire.dropout_vector / self.inputWire.dropout_proba
        return self.outputs

    def relu(self, x):
        # Calculate sigmoid on vector cells
        self.relu_values = np.maximum(0, x)
        return self.relu_values

    def backward(self):
        d_p_wrt_f = np.zeros(self.relu_values.shape)
        d_p_wrt_f[self.relu_values > 0] = 1
        if self.dropout:
            d_p_wrt_f = d_p_wrt_f * self.inputWire.dropout_vector
        self.d_E_wrt_f = np.sum(self.outputWire.gradients[1:], 1) * d_p_wrt_f
        d_E_wrt_x = self.inputWire.weights.dot(
            np.diag(self.d_E_wrt_f))  # Multiply outgoing gradients by the dE/df of their respective node in this layer
        self.inputWire.gradients = d_E_wrt_x
        # print("backward gradient:", d_E_wrt_x)

        self.update()

    def update(self):
        global alpha
        # if self.dropout:
        #     self.inputWire.weights -= alpha*(np.outer(self.inputs, self.inputWire.dropout_matrix.dot(self.d_E_wrt_f)))
        # self.inputWire.velocity = self.inputWire.momentum*self.inputWire.velocity - alpha * (np.outer(self.inputs, self.d_E_wrt_f))
        # self.inputWire.weights += self.inputWire.velocity
        self.inputWire.weights -= alpha * (np.outer(self.inputs, self.d_E_wrt_f))


class softmaxLayer( Layer ):
    def __init__( self, inputWire, units, dropout=False):
        super().__init__( inputWire, units, dropout )

    def forward(self, x, training=False):
        self.inputs = x
        # Add input value of "1" to the input array for the bias weight (w_0*x_0 + ...)
        self.inputs = np.insert(x, 0, 1, 0)
        self.outputs = self.softmax(self.inputWire.weights.T.dot(self.inputs))
        return self.outputs

    def softmax( self, x ):
        # Calculate sigmoid on vector cells
        exponentials = np.exp(x - np.max(x))
        self.softmax_values = exponentials / np.sum(exponentials)
        return self.softmax_values
        
    def backward( self ):
        self.inputWire.gradients = None

        d_p_wrt_f = self.softmax_values * (1 - self.softmax_values)
        self.d_E_wrt_f = self.outputWire.gradients * d_p_wrt_f # single incoming gradient
        d_E_wrt_x = self.inputWire.weights.dot(np.diag(self.d_E_wrt_f)) # Multiply outgoing gradients by the dE/df of their respective node in this layer
        self.inputWire.gradients = d_E_wrt_x

        self.update()

    def backward( self, y):
        self.inputWire.gradients = None

        self.d_E_wrt_f = self.outputs - y # single incoming gradient
        d_E_wrt_x = self.inputWire.weights.dot(np.diag(self.d_E_wrt_f)) # Multiply outgoing gradients by the dE/df of their respective node in this layer
        self.inputWire.gradients = d_E_wrt_x

        self.update()

    def update( self ):
        global alpha
        # TODO: Refactor away from the outer() function to allow for mini-batch
        self.inputWire.weights -= alpha*(np.outer(self.inputs, self.d_E_wrt_f))

In [106]:
from numpy.random import multivariate_normal

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return [a[p], b[p]]

In [107]:
mean_w1 = np.array([0, 0])
covar_w1 = np.array([[1,0], [0,1]])
mean_w2 = np.array([1, 0.5])
covar_w2 = np.array([[3, 1],[1, 2]])

# Generate training data and labels
training_w1_data = multivariate_normal(mean_w1, covar_w1, size=(50))
training_w1_labels = np.zeros(training_w1_data.shape[0])
training_w2_data = multivariate_normal(mean_w2, covar_w2, size=(50))
training_w2_labels = np.ones(training_w2_data.shape[0])

# Generate test data and labels
test_w1_data = multivariate_normal(mean_w1, covar_w1, size=(20))
test_w1_labels = np.zeros(test_w1_data.shape[0])
test_w1_labels = test_w1_labels.reshape((-1, 1))
test_w2_data = multivariate_normal(mean_w2, covar_w2, size=(20))
test_w2_labels = np.ones(test_w2_data.shape[0])
test_w2_labels = test_w2_labels.reshape((-1, 1))
training_data = np.r_[training_w1_data, training_w2_data]
training_labels = np.r_[training_w1_labels, training_w2_labels]
training = unison_shuffled_copies(training_data, training_labels)
training[1] = training[1].reshape((-1, 1))
test_data = np.r_[test_w1_data, test_w2_data]
test_labels = np.r_[test_w1_labels, test_w2_labels]
test = unison_shuffled_copies(test_data, test_labels)
test[1] = test[1].reshape((-1, 1))


In [108]:
training

[array([[ 2.58526154,  1.50673691],
        [ 0.9459079 ,  0.78428377],
        [-3.06450546,  0.50713284],
        [ 2.42758543, -0.35320139],
        [-2.88129556, -1.9514777 ],
        [ 2.32506983, -0.28630141],
        [-1.46409696, -2.27279316],
        [ 2.18815404,  0.48744455],
        [-1.4812801 , -0.74767901],
        [ 0.01544835,  1.07868754],
        [-2.93398436, -0.80002641],
        [ 2.14113948, -2.21062508],
        [ 3.49510598,  0.79579375],
        [-2.17283067, -1.49347133],
        [ 1.26745834,  1.16406692],
        [ 0.3226758 , -0.48508581],
        [-2.08800556,  0.28749216],
        [-0.04611488, -1.44179358],
        [ 1.55772397, -0.44909358],
        [ 1.16245024, -0.69844262],
        [ 2.43211562,  0.32326882],
        [ 2.3920471 , -0.07776911],
        [ 0.5244444 , -0.83032409],
        [-3.22597374,  0.86458153],
        [ 0.4103365 ,  1.0874075 ],
        [ 0.23133508,  0.20605669],
        [ 1.40455074,  0.78396968],
        [-1.37722745,  1.421

In [111]:
test

[array([[-2.48410575, -1.97644187],
        [ 0.13786899,  0.99134267],
        [-0.28984828, -0.2647309 ],
        [ 2.7511627 ,  0.67158909],
        [-1.09041331,  0.78303934],
        [-2.59191661,  0.60833617],
        [ 4.76138396,  4.0574443 ],
        [-1.22032617, -0.89762013],
        [ 1.63162746,  0.96254078],
        [ 3.89733431,  0.56318168],
        [-1.19883503, -2.34908951],
        [ 0.29675446, -0.01161843],
        [-0.5471955 , -0.28957303],
        [ 1.39961336,  0.76535059],
        [ 0.38487213, -0.76017354],
        [ 1.61579949,  1.69676901],
        [-0.98058741, -1.10457408],
        [ 3.29698373,  0.88174353],
        [ 2.79805045,  0.58797784],
        [ 0.7607794 ,  0.18260414],
        [-0.49255052, -0.92581729],
        [ 0.69187397, -0.0375058 ],
        [ 1.16601016,  0.48491364],
        [ 1.43114001,  2.12053659],
        [ 0.20666562, -2.78109737],
        [ 0.98481548, -0.56408926],
        [ 1.75143071,  1.26361796],
        [ 0.14770322,  0.870

In [112]:
EPOCHS = 20
nn_architecture = [("input", 2), ("sigmoid", 10), ("sigmoid", 1)]
nn = NeuralNetwork(nn_architecture, dropout=True)
nn.train_and_validate(training, test, EPOCHS)
nn.saveModel()

print("Unpickling model and revalidating")
nn2 = nn.load_model()
nn2.test(test)

Epoch: 0
training output: [ 0.71272588] label: [ 0.] errorGradient: [-1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.53214286] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.53981902] label: [ 0.] errorGradient: [-1.]
training output: [ 1.81094701] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.38923783] label: [ 0.] errorGradient: [-1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.61438419] label: [

training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.25847356] label: [ 1.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.90784324] label: [ 0.] errorGradient: [-1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.77444021] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.96061956] label: [ 1.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.11847221] label: [ 1.] errorGradient: [ 0.]
training output: [ 1.13956303] label: [ 1.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.10068491] label: [ 1.] errorGradient: [ 0.]
training output: [ 0.89985591] label: [ 1.] errorGradient: [ 0.]
training output: [ 1.236

training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.99113501] label: [ 1.] errorGradient: [ 0.]
training output: [ 1.74427599] label: [ 1.] errorGradient: [-1.]
training output: [ 1.68088871] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.61405279] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.22010483] label: [ 0.] errorGradient: [-1.]
training output: [ 1.39542224] label: [ 0.] errorGradient: [-1.]
training output: [ 1.49973739] label: [ 0.] errorGradient: [-1.]
training output: [ 1.19608844] label: [ 1.] errorGradient: [ 0.]
training output: [ 1.88263201] label: [ 1.] errorGradient: [-1.]
training output: [ 1.02753023] label: [ 0.] errorGradient: [-1.]
training

training output: [ 1.78377691] label: [ 1.] errorGradient: [-1.]
training output: [ 1.91740114] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.42053353] label: [ 1.] errorGradient: [ 0.]
training output: [ 1.39711959] label: [ 1.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.45712234] label: [ 1.] errorGradient: [ 0.]
training output: [ 1.24296674] label: [ 0.] errorGradient: [-1.]
training output: [ 1.20691101] label: [ 0.] errorGradient: [-1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.65136271] label: [ 1.] errorGradient: [-1.]
training output: [ 1.55631918] label: [ 1.] errorGradient: [-1.]
training output: [ 1.60323952] label: [ 1.] errorGradient: [-1.]
training

[ 0.13786899  0.99134267] [ 0.85848254] [ 1.]
[-0.28984828 -0.2647309 ] [ 0.88359085] [ 1.]
[ 2.7511627   0.67158909] [ 0.84766126] [ 0.]
[-1.09041331  0.78303934] [ 0.87897921] [ 0.]
[-2.59191661  0.60833617] [ 0.89900648] [ 1.]
[ 4.76138396  4.0574443 ] [ 0.80309653] [ 1.]
[-1.22032617 -0.89762013] [ 0.90695674] [ 0.]
[ 1.63162746  0.96254078] [ 0.84434552] [ 0.]
[ 3.89733431  0.56318168] [ 0.84771282] [ 1.]
[-1.19883503 -2.34908951] [ 0.91679163] [ 0.]
[ 0.29675446 -0.01161843] [ 0.87106876] [ 0.]
[-0.5471955  -0.28957303] [ 0.88774888] [ 0.]
[ 1.39961336  0.76535059] [ 0.84940214] [ 0.]
[ 0.38487213 -0.76017354] [ 0.87869303] [ 1.]
[ 1.61579949  1.69676901] [ 0.83155301] [ 1.]
[-0.98058741 -1.10457408] [ 0.90696461] [ 1.]
[ 3.29698373  0.88174353] [ 0.84531089] [ 0.]
[ 2.79805045  0.58797784] [ 0.84861038] [ 0.]
[ 0.7607794   0.18260414] [ 0.86326958] [ 1.]
[-0.49255052 -0.92581729] [ 0.89715359] [ 1.]
[ 0.69187397 -0.0375058 ] [ 0.86693652] [ 0.]
[ 1.16601016  0.48491364] [ 0.8557

...

training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.81487145] label: [ 0.] errorGradient: [-2.]
[-2.48410575 -1.97644187] [ 0.96973962] [ 0.]
[ 0.13786899  0.99134267] [ 0.92955342] [ 1.]
[-0.28984828 -0.2647309 ] [ 0.94459629] [ 1.]
[ 2.7511627   0.67158909] [ 0.90552856] [ 0.]
[-1.09041331  0.78303934] [ 0.94735692] [ 0.]
[-2.59191661  0.60833617] [ 0.95987689] [ 1.]
[ 4.76138396  4.0574443 ] [ 0.8703158] [ 1.]
[-1.22032617 -0.89762013] [ 0.95868795] [ 0.]
[ 1.63162746  0.96254078] [ 0.90957466] [ 0.]
[ 3.89733

training output: [ 1.83894595] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.97417531] label: [ 0.] errorGradient: [-2.]
training output: [ 1.94777105] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.95249022] label: [ 1.] errorGradient: [-1.]
training output: [ 1.81425422] label: [ 0.] errorGradient: [-2.]
training output: [ 1.79619381] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.75516654] label: [ 1.] errorGradient: [-1.]
training output: [ 1.97985727] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.7705249] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: 

training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.82097614] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.56855468] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.9781386] label: [ 1.] errorGradient: [-1.]
training output: [ 1.97268143] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.84454442] label: [ 1.] errorGradient: [-1.]
training output: [ 1.8584612] label: [ 0.] errorGradient: [-2.]
training output: [ 1.93363215] label: [ 1.

training output: [ 1.77142768] label: [ 0.] errorGradient: [-2.]
training output: [ 1.96268128] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.81595035] label: [ 0.] errorGradient: [-2.]
training output: [ 1.99270124] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.76430463] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.92808216] label: [ 1.] errorGradient: [-1.]
training output: [ 1.97040033] label: [ 1.] errorGradient: [-1.]
training output: [ 1.81930427] label: [ 0.] errorGradient: [-2.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 0.] l

training output: [ 1.95655821] label: [ 0.] errorGradient: [-2.]
training output: [ 1.91403557] label: [ 1.] errorGradient: [-1.]
training output: [ 1.98823033] label: [ 1.] errorGradient: [-1.]
training output: [ 1.97182521] label: [ 0.] errorGradient: [-2.]
training output: [ 1.92270733] label: [ 1.] errorGradient: [-1.]
training output: [ 1.97478525] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.95049072] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.93816828] label: [ 0.] errorGradient: [-2.]
training output: [ 1.89510148] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 0.] errorGradient: [ 0.]
training output: [ 1.96991848] label: [ 0.] errorGradient: [-2.]
training output: [ 1.94150198] label: [ 1.] errorGradient: [-1.]
training output: [ 0.] label: [ 1.] errorGradient: [ 1.]
training output: [ 1.92310752] label: [ 1.] errorGradient